In [ ]:
from scannerpy import Client, DeviceType
from scannerpy.storage import NamedVideoStream, NamedStream
from query.models import Video
from scannertools import maskrcnn_detection, densepose_detection

from esper.table_tennis.utils import *
from esper.widget import *

import os
import sys
import numpy as np

In [ ]:
movie_path = '/app/data/videos/sample-clip.mp4'
movie_name = os.path.splitext(os.path.basename(movie_path))[0]
video = Video.objects.filter(path__contains='sample')[0]

In [ ]:
video = Video.objects.filter(path__contains="men_single")[0]
movie_path = video.path
movie_name = video.item_name()
movie_name

In [ ]:
sc = Client()
stride = 1
input_stream = NamedVideoStream(sc, movie_name, path=movie_path)
frame = sc.io.Input([input_stream])
strided_frame = sc.streams.Stride(frame, [stride])

In [ ]:
s = sc.summarize()
print(s)

In [ ]:
sc.delete_table("Tabletennis_2012_Olympics_men_single_final_bronze.mp4_maskrcnn")

# MaskRCNN

In [ ]:
maskrcnn_stream = NamedStream(sc, movie_name + '_maskrcnn')

## Visualize single

In [ ]:
fid = 25000
seq = sc.sequence(maskrcnn_stream._name)
obj = seq.load(workers=1, rows=[fid])

In [ ]:
metadata = next(obj)
frame = load_frame(video, fid, [])
result = maskrcnn_detection.visualize_one_image(frame, metadata, min_score_thresh=0.5)
imshow(result)

## Visualize montage

In [ ]:
images = []
for i, obj in tqdm(enumerate(seq.load(workers=1))):
    if i % 1000 == 0:
        print(i)
    if i < 20 * 60 * 25:
        continue
    if i % 100 == 0:
        frame = load_frame(video, i, [])
        result = maskrcnn_detection.visualize_one_image(frame, obj, min_score_thresh=0.7)
        images.append(result)
    if len(images) > 100:
        break

In [ ]:
create_montage_from_images(images[:100], output_path='/app/result/maskrcnn_result1.jpg',
                          width=2160, num_cols=10)

# DensePose

In [ ]:
densepose_stream = NamedStream(sc, movie_name + '_densepose')

In [ ]:
fid = 100
seq = sc.sequence(densepose_stream._name)
obj = seq.load(workers=1, rows=[fid])
metadata = next(obj)

In [ ]:
frame = load_frame(video, fid, [])
result = densepose_detection.visualize_one_image(frame, metadata, min_score_thresh=0.7, show_keypoint=True, show_box=True)
imshow(result)